<a href="https://colab.research.google.com/github/sanjay-thiyagarajan/score-predictor/blob/main/CricPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://internalapp.nptel.ac.in/contest/reference_docs/ipl_csv2.zip'
!mkdir '/content/dataset'
!unzip '/content/ipl_csv2.zip' -d '/content/dataset'

In [2]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [25]:
train_parent = []
DATASET_DIR = '/content/dataset/'
for file in os.listdir(DATASET_DIR):
  filename = os.path.join(DATASET_DIR, file)
  df = pd.read_csv(filename, error_bad_lines=False, header=0)
  train_parent.append(df)
train_parent = pd.concat(train_parent, axis=0, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 7: expected 1 fields, saw 3\nSkipping line 10: expected 1 fields, saw 2\nSkipping line 13: expected 1 fields, saw 3\nSkipping line 26: expected 1 fields, saw 2\nSkipping line 29: expected 1 fields, saw 2\nSkipping line 30: expected 1 fields, saw 2\nSkipping line 33: expected 1 fields, saw 2\nSkipping line 37: expected 1 fields, saw 2\nSkipping line 38: expected 1 fields, saw 2\nSkipping line 39: expected 1 fields, saw 2\nSkipping line 42: expected 1 fields, saw 4\nSkipping line 47: expected 1 fields, saw 2\nSkipping line 74: expected 1 fields, saw 4\nSkipping line 78: expected 1 fields, saw 3\nSkipping line 81: expected 1 fields, saw 2\nSkipping line 84: expected 1 fields, saw 5\nSkipping line 85: expected 1 fields, saw 2\nSkipping 

In [26]:
train_parent = train_parent.drop(columns=['All Indian Premier League match data in CSV format'])
d = {'runs_off_bat': 'sum'}
train_parent = train_parent.groupby(['match_id','innings', 'striker', 'bowler'], as_index=False).aggregate(d)
train_parent.drop(columns=['match_id', 'innings'])

,striker,bowler,runs_off_bat
0,BB McCullum,AA Noffke,48.0
1,BB McCullum,CL White,32.0
2,BB McCullum,JH Kallis,64.0
3,BB McCullum,P Kumar,64.0
4,BB McCullum,SB Joshi,42.0
...,...,...,...
40098,Mohammad Nabi,Shakib Al Hasan,4.0
40099,V Shankar,AD Russell,6.0
40100,V Shankar,CV Varun,16.0
40101,WP Saha,Harbhajan Singh,14.0


# **Checking with limited features**

In [ ]:
featured_df = train_parent[['striker', 'runs_off_bat', 'bowler']]
featured_df = featured_df.dropna()
people_encoder = LabelEncoder()
people = list(np.array(featured_df[['striker']]).flatten()) + list(np.array(featured_df[['bowler']]).flatten())
people = list(set(people))
people_encoder.fit(people)
featured_df[['striker']] = pd.DataFrame(np.array(people_encoder.transform(featured_df[['striker']]), dtype='int32'))
featured_df[['bowler']] = pd.DataFrame(np.array(people_encoder.transform(featured_df[['bowler']]), dtype='int32'))
featured_df = featured_df.dropna()
featured_df

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,striker,runs_off_bat,bowler
0,331.0,0.0,1.0
1,331.0,0.0,1.0
2,331.0,0.0,1.0
3,474.0,0.0,1.0
4,474.0,0.0,1.0
...,...,...,...
388703,551.0,1.0,25.0
388704,551.0,1.0,25.0
388705,551.0,1.0,25.0
388706,157.0,1.0,25.0


In [ ]:
X_train = np.array(featured_df.drop(columns=['runs_off_bat']))
Y_train = np.array(featured_df[['runs_off_bat']])
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
sum(regressor.predict(X_train))

array([481306.99999996])

In [ ]:
sum(Y_train)

array([481307.])